We use CNN based neural network to classify our Pneumonia train images.

In [18]:
!git clone https://github.com/tensorflow/models.git

import sys
sys.path.append('/kaggle/working/models/research/object_detection/utils')
sys.path.append('/kaggle/working/models/research/object_detection/dataset_tools')

Cloning into 'models'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 23413 (delta 33), reused 38 (delta 21), pack-reused 23359
Receiving objects: 100% (23413/23413), 563.02 MiB | 33.25 MiB/s, done.
Resolving deltas: 100% (13693/13693), done.
Checking out files: 100% (2797/2797), done.


In [26]:
import tensorflow as tf

from tensorflow.models.research.object_detection.utils import dataset_util

import pandas as pd
import pydicom

from io import BytesIO

import numpy as np

flags = tf.app.flags
FLAGS = flags.FLAGS

debug = False

def create_tf_example(patientId, boxes):
    height = 1024 # Image height
    width = 1024 # Image width

    path = "../../all/stage_2_train_images/" + patientId + ".dcm"

    ds = pydicom.dcmread(path)

    filename = bytes(patientId + '.jpg', 'utf-8') # Filename of the image. Empty if image is not from file
    image_format = b'jpeg' # b'jpeg' or b'png'

    encoded_image_data = ds.PixelData[16:]
    if (debug):
        print(encoded_image_data[:3])

    xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [] # List of normalized right x coordinates in bounding box
                # (1 per box)
    ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [] # List of normalized bottom y coordinates in bounding box
                # (1 per box)

    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = [] # List of integer class id of bounding box (1 per box)

    for box in boxes:
        if not np.isnan(box[0]):
            if (debug):
                print(box)
            classes_text.append(b'pneumonia')
            classes.append(1)
            
            # x-min y-min width height
            xmins.append(box[0] / width)   # store normalized values for bbox
            xmaxs.append((box[0] + box[2]) / width)
            ymins.append(box[1] / height)
            ymaxs.append((box[1] + box[3]) / height)

    if (debug):
        print(xmins)
        print(xmaxs)
        print(ymins)
        print(ymaxs)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

import contextlib2
from tensorflow.models.research.object_detection.dataset_tools import tf_record_creation_util

num_shards=10
output_filebase='train'

with contextlib2.ExitStack() as tf_record_close_stack:
    output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(tf_record_close_stack, output_filebase, num_shards)


    train = pd.read_csv("../../all/stage_2_train_labels.csv")
    groups = train.groupby('patientId')

    count = 0

    for patientId in train.drop_duplicates('patientId')['patientId']:
        print('[{c}]processing patientId={p}'.format(c=count,p=patientId))

        boxes = groups.get_group(patientId).drop(columns=['patientId']).as_matrix()
        tf_example = create_tf_example(patientId, boxes)

        output_shard_index = count % num_shards
        output_tfrecords[output_shard_index].write(tf_example.SerializeToString())

        count += 1

[0]processing patientId=0004cfab-14fd-4e49-80ba-63a80b6bddd6
[1]processing patientId=00313ee0-9eaa-42f4-b0ab-c148ed3241cd
[2]processing patientId=00322d4d-1c29-4943-afc9-b6754be640eb
[3]processing patientId=003d8fa0-6bf1-40ed-b54c-ac657f8495c5
[4]processing patientId=00436515-870c-4b36-a041-de91049b9ab4
[5]processing patientId=00569f44-917d-4c86-a842-81832af98c30
[6]processing patientId=006cec2e-6ce2-4549-bffa-eadfcd1e9970
[7]processing patientId=00704310-78a8-4b38-8475-49f4573b2dbb
[8]processing patientId=008c19e8-a820-403a-930a-bc74a4053664
[9]processing patientId=009482dc-3db5-48d4-8580-5c89c4f01334
[10]processing patientId=009eb222-eabc-4150-8121-d5a6d06b8ebf
[11]processing patientId=00a85be6-6eb0-421d-8acf-ff2dc0007e8a
[12]processing patientId=00aecb01-a116-45a2-956c-08d2fa55433f
[13]processing patientId=00c0b293-48e7-4e16-ac76-9269ba535a62
[14]processing patientId=00d7c36e-3cdf-4df6-ac03-6c30cdc8e85b
[15]processing patientId=00f08de1-517e-4652-a04f-d1dc9ee48593
[16]processing pat

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:95: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


[23]processing patientId=01538c3b-3de9-4fbb-95a8-732235821bbf
[24]processing patientId=016b1f90-bb9a-4d3a-9c38-74af5fffd5b5
[25]processing patientId=0174c4bb-28f5-41e3-a13f-a396badc18bd
[26]processing patientId=017c7b5b-618e-4bc9-943c-04c6a988d992
[27]processing patientId=019ca122-9cdf-4704-b7a9-449c8a1c263e
[28]processing patientId=019d950b-dd38-4cf3-a686-527a75728be6
[29]processing patientId=019e035e-2f82-4c66-a198-57422a27925f
[30]processing patientId=01a4059c-22f7-4f51-8a27-50aff0b3aeb3
[31]processing patientId=01a5594f-e5d4-4f7a-b79d-3f57559fe37b
[32]processing patientId=01a6eaa6-222f-4ea8-9874-bbd89dc1a1ce
[33]processing patientId=01a7353d-25bb-4ff8-916b-f50dd541dccf
[34]processing patientId=01aad2a6-3b93-45e3-bf37-2d73348cb6fc
[35]processing patientId=01adfd2f-7bc7-4cef-ab68-a0992752b620
[36]processing patientId=01b15f07-1149-4ff8-9756-bc821e41b97c
[37]processing patientId=01b56434-4dd9-4994-bcc5-0b70a36e415a
[38]processing patientId=01b9e362-4950-40f5-88fa-7557ac2a45bb
[39]proc

[183]processing patientId=061bb9bf-13c6-4ea0-bd82-5768fb83dff8
[184]processing patientId=061ccabd-b0cf-4238-a053-28a1960c1923
[185]processing patientId=061d0ff4-a611-42ce-b780-1d1eb55e50ab
[186]processing patientId=0620afa3-b2ea-442e-9e4f-c854086ca763
[187]processing patientId=06216b5c-c03f-4562-bfec-bec5235943d2
[188]processing patientId=062374a3-6ca1-48d2-aa3c-820053773b54
[189]processing patientId=0623ea89-c89c-4743-b492-55e398de4c9b
[190]processing patientId=06250f9d-5ad9-44ca-9283-dbf787e2ad22
[191]processing patientId=06327569-4e4f-4fdc-a116-db367c079669
[192]processing patientId=0634d937-1be6-475e-a5fd-dc9a2b6a8da6
[193]processing patientId=0637b804-e693-48d1-8716-f7c63ce6162a
[194]processing patientId=0638126f-126f-4f53-befe-538ff87597c6
[195]processing patientId=0638709c-d31d-4a59-a9e4-fc408ae36145
[196]processing patientId=063a4fc6-ef5d-4370-8b0b-c2752a4c905c
[197]processing patientId=063f63c9-7e5b-4360-8d19-0060d849fdbc
[198]processing patientId=063fed6c-4ecd-4a74-8f0d-c7a90

[350]processing patientId=078aecd3-abe5-4a09-9c2c-4b13f852bd41
[351]processing patientId=0790ee1e-70c5-4f8f-9b80-cd10ecfd90f2
[352]processing patientId=079269e0-96f3-4964-8def-b13bd0aa837a
[353]processing patientId=0793267b-ba31-4f77-bee8-acde7a571231
[354]processing patientId=0796d0a6-8aa5-4edf-9c82-33523d8da0c2
[355]processing patientId=0798f0c3-bf7d-4f04-bd5e-b119163ea1b0
[356]processing patientId=079f6a17-f9ce-48b2-a6dc-2a83eb07edf7
[357]processing patientId=079fa565-0c4e-46a2-9a3c-f6341d97fd5c
[358]processing patientId=07a16de4-9d4c-4e88-a980-37734b8a4cdd
[359]processing patientId=07a35aa8-491e-49f2-a9a3-13df10f32484
[360]processing patientId=07a750fe-01d7-4d49-96c6-06fc1cba8504
[361]processing patientId=07a7adcc-5911-46a0-acc1-aaac588c0245
[362]processing patientId=07abb49d-c23c-41a5-b38a-b093011b59dd
[363]processing patientId=07aeb82e-773b-4498-95c0-fabdf4985bb2
[364]processing patientId=07afb190-e345-418c-8bd8-ca864b11b6d1
[365]processing patientId=07b24c8e-be16-4d5d-992e-b66e7

...

[25913]processing patientId=100d2b25-aba5-4502-bd2a-edeaa66fde00
[25914]processing patientId=100fb3d8-99c5-4723-9385-11b236f9aae4
[25915]processing patientId=100ffc09-4644-4936-95eb-54c41bdeb816
[25916]processing patientId=1013ded6-9505-4e1b-a923-f263ad013d0e
[25917]processing patientId=10155593-ee63-4405-861a-81ef41ddeb56
[25918]processing patientId=1015e103-bda6-4664-978d-f761cd2982cb
[25919]processing patientId=10163e08-1de2-44e1-9145-1922f4dd6805
[25920]processing patientId=10178701-ea33-486c-87f3-e52977d83266
[25921]processing patientId=101921f9-4651-41e1-b7dc-8e0124d5a5d1
[25922]processing patientId=101e82b5-8be1-4318-84c9-0c64813f9ac5
[25923]processing patientId=103451bb-1f8c-4209-abce-0be7e9a6c8fb
[25924]processing patientId=10442f49-c354-44f8-8ca2-a4652713285a
[25925]processing patientId=106d08b7-b1b7-4ca1-be8d-8de94d2ef682
[25926]processing patientId=106e39b9-9ca8-4437-b7f9-f114fd3aec61
[25927]processing patientId=1079ba63-0681-45b7-b1da-fe11ede2b9ae
[25928]processing patient

[26077]processing patientId=1aea68e3-a8b7-4402-92a5-16a1f23c6d4a
[26078]processing patientId=1aeb19e3-7365-4d66-afef-5157a3aca280
[26079]processing patientId=1af01c2b-ade7-4482-a406-4e1c24335f1a
[26080]processing patientId=1af50916-9b55-4df5-b1d3-dde1af2bc441
[26081]processing patientId=1b0b50d0-c31e-4713-b252-292423149457
[26082]processing patientId=1b0b9b81-dc8b-4b22-aaea-1a985009a522
[26083]processing patientId=1b14f17b-bfca-4ed6-9f7f-9fe8e0b4a007
[26084]processing patientId=1b167f33-a18a-4aab-8a29-2a9d1d41c054
[26085]processing patientId=1b191289-3e02-4521-b96d-4d412d02c102
[26086]processing patientId=1b1dc237-cefb-48f9-87ae-467e99fa9cf0
[26087]processing patientId=1b26e086-834d-49ec-b0fc-d2fafccb4aa6
[26088]processing patientId=1b2fcebf-3743-40ef-92e7-e0802c77a5c2
[26089]processing patientId=1b440724-2e24-4124-b7c7-0bc82015a674
[26090]processing patientId=1b4d9ebb-644e-4602-ad36-d83cf7415be6
[26091]processing patientId=1b52079c-b9ef-42d3-9e9a-17aea3bc6031
[26092]processing patient

[26242]processing patientId=21cbfa1b-6faa-4356-b1eb-a180d7d9b4ac
[26243]processing patientId=21cca2a5-6cb9-4667-bde6-5fe4ac8d2a29
[26244]processing patientId=220e0278-363d-431b-8094-1e28085ddb0f
[26245]processing patientId=2232fe6c-e152-4875-9b6d-1706a3b1652c
[26246]processing patientId=223d7974-cb7f-47bb-9502-d91b363153dc
[26247]processing patientId=224a054f-8497-4e62-b4fe-b7829c62879e
[26248]processing patientId=224c3686-d481-46f9-8a3a-4e492ba3363f
[26249]processing patientId=2263b413-900d-4c5a-99cc-bb551a39f131
[26250]processing patientId=22956af6-6152-4458-a39d-a45706b8a96c
[26251]processing patientId=229c584f-7174-4fcf-b740-7127cbf53525
[26252]processing patientId=22a39d0b-3a66-400a-96cf-97a228a2de79
[26253]processing patientId=22ce653c-e7f4-45d5-9c5e-4abdc753b1b0
[26254]processing patientId=22d23df8-22af-410c-bea9-2e6e8eb1ce94
[26255]processing patientId=22da0004-e69d-49c5-bde5-252ab5895da0
[26256]processing patientId=22e76fb4-c74e-4cc3-820a-832a4467dac6
[26257]processing patient

[26407]processing patientId=28d0f2da-cacb-43b0-848d-4c6389d55d99
[26408]processing patientId=28dfbd38-b24a-4cfb-a355-34c226dd2aa2
[26409]processing patientId=28eaff00-5fb7-40ee-8206-9d2ff5b081e0
[26410]processing patientId=28f1790b-5643-4db3-9d4d-de30d4784a0c
[26411]processing patientId=28fa453d-027a-4569-9e22-7814e7b74f43
[26412]processing patientId=29000617-ac33-40a4-ac5c-19fb586edb69
[26413]processing patientId=2902bb15-a15a-4203-8a4b-6dca00a3d9fa
[26414]processing patientId=290ba340-b0c0-44d5-ab13-e0e315504cac
[26415]processing patientId=290e32e9-3632-4642-a1d4-70c975d376b7
[26416]processing patientId=29128fda-79ce-4a14-bbf9-9884422075d0
[26417]processing patientId=292a1c21-012d-4039-b95e-02524e82bce1
[26418]processing patientId=292aa643-7569-4ef7-8a51-12b62440a2b0
[26419]processing patientId=293f4b7b-3a6e-44a6-b9d2-810b139094cf
[26420]processing patientId=294ffc42-a92a-40d6-bcda-96bb3c9b8def
[26421]processing patientId=295de041-5711-400a-8241-7d7cf500971e
[26422]processing patient

[26573]processing patientId=2f6ac842-e243-4c5f-a48c-d70862e316d5
[26574]processing patientId=2f6debad-7dac-4feb-beaf-61f75ec38eae
[26575]processing patientId=2f7fd25d-4968-44bb-96c3-91e377284546
[26576]processing patientId=2f80ae68-fdc8-4e4c-9e64-7de9db969518
[26577]processing patientId=2f876297-aa92-457a-bc28-efa539885b93
[26578]processing patientId=2f98447b-c43a-4dc2-873d-c326a34c1ba2
[26579]processing patientId=2f9c1b97-74a7-4781-bad3-c46cc08c69c8
[26580]processing patientId=2fa75092-e364-4326-b5b0-f1d61df464c8
[26581]processing patientId=2fa8e28a-dce6-4c68-8bb6-b2537476577d
[26582]processing patientId=2faeab4f-17bf-410e-99b9-77bf4ae75963
[26583]processing patientId=2fb8f201-843f-44fd-90cd-d1f257f996cf
[26584]processing patientId=2fc6c353-a51d-4eb7-ae51-d971f12a38d9
[26585]processing patientId=2fd675cd-8340-498c-90a2-64f5e9abf7b3
[26586]processing patientId=2fd90436-1b1e-4352-abe6-ea41ec5df91a
[26587]processing patientId=2ff18655-f28c-4992-a622-44300e071629
[26588]processing patient

In [27]:
# clean up names a bit and take the first as validation
!mkdir train
!mkdir val

# TODO: use train_test_split to do validation rows
!mv train-00000-of-00010 val/val-00001-of-00001

!mv train-00001-of-00010 train/train-00001-of-00009
!mv train-00002-of-00010 train/train-00002-of-00009
!mv train-00003-of-00010 train/train-00003-of-00009
!mv train-00004-of-00010 train/train-00004-of-00009
!mv train-00005-of-00010 train/train-00005-of-00009
!mv train-00006-of-00010 train/train-00006-of-00009
!mv train-00007-of-00010 train/train-00007-of-00009
!mv train-00008-of-00010 train/train-00008-of-00009
!mv train-00009-of-00010 train/train-00009-of-00009

In [28]:
# remove the models git repo
!rm -rf models